## 6日目
### for 構文、再び
Haskell の do 記法と Scala の for 構文には微妙な違いがある

do 表記の例

```haskell
foo = do
  x <- Just 3
  y <- Just "!"
  Just (show x ++ y)
```

In [1]:
import scalaz._, Scalaz._

import scalaz._, Scalaz._

In [2]:
def foo = for {
    x <- 3.some
    y <- "!".some
} yield x.shows + y

defined function foo

* Haskell はモナディックな値を最後に返す
* Scala は素の値を yield が強制的にコンテキストに入れている

```haskell
in3 start = do
  first <- moveKnight start
  second <- moveKnight first
  moveKnight second
```

* Scala では moveKnight second の値を抽出して yield で再包装せずに書くことができない

```scala
def in3: List[KnightPos] = for {
    first <- move
    second <- first.move
    third <- second.move
} yield third
```

### Writer? 中の人なんていません!
* Maybeモナドが失敗の可能性という文脈付きの値を表し、リストモナドが非決定性がついた値を表しているのに対し、Writerモナドは、もう１つの値がくっついた値を表し、付加された値はログのように振舞います。

applyLog関数を実装してみよう

In [3]:
def isBigGang(x: Int): (Boolean, String) =
  (x > 9, "Compared gang size to 9.")

defined function isBigGang

In [4]:
implicit class PairOps[A](pair: (A, String)) {
    def applyLog[B](f: A => (B, String)): (B, String) = {
        val (x, log) = pair
        val (y, newlog) = f(x)
        (y, log ++ newlog)
    }
}

defined class PairOps

In [5]:
(3, "Smallish gang.") applyLog isBigGang

res4: (Boolean, String) = (false, "Smallish gang.Compared gang size to 9.")

In [6]:
implicit class PairOps[A, B: Monoid](pair: (A, B)) {
    def applyLog[C](f: A => (C, B)): (C, B) = {
        val (x, log) = pair
        val (y, newlog) = f(x)
        (y, log |+| newlog)
    }
}

defined class PairOps

In [7]:
(3, "Smallish gang.") applyLog isBigGang

res6: (Boolean, String) = (false, "Smallish gang.Compared gang size to 9.")

### Writer
* 値にモノイドのオマケをつけるには、タプルに入れるだけです。
* Writer w a 型の実態は、そんなタプルの newtype ラッパに過ぎず、定義はとてもシンプルです。

Scalaz でこれに対応するのは Writer

```scala
type Writer[+W, +A] = WriterT[Id, W, A]
```

WriterTを単純化したもの

```scala
sealed trait WriterT[F[+_], +W, +A] { self =>
    val run: F[(W, A)]
    
    def written(implicit F: Functor[F]): F[W] =
      F.map(run)(_._1)
    def value(implicit F: Functor[F]): F[A] =
      F.map(run)(_._2)
}
```

In [8]:
3.set("Smallish gang.")

res7: Writer[String, Int] = WriterT(("Smallish gang.", 3))

`import Scalaz._` によって全てのデータ型に対して以下の演算子が導入されるらしい

```scala
trait ToDataOps extends ToIdOps with ToTreeOps with ToWriterOps with ToValidationOps with ToReducerOps with ToKleisliOps
```

件の演算子は WriterOpsの一部

```scala
final class WriterOps[A](self: A) {
    def set[W](w: W): Writer[W, A] = WriterT.writer(w -> self)
    
    def tell: Writer[A, Unit] = WriterT.tell(self)
}
```

In [9]:
3.set("something")

res8: Writer[String, Int] = WriterT(("something", 3))

In [10]:
"something".tell

res9: Writer[String, Unit] = WriterT(("something", ()))

In [10]:
scalaz.MonadTell[scalaz.Writer, String]

cmd10.sc:1: scalaz.Writer takes two type parameters, expected: one
val res10 = scalaz.MonadTell[scalaz.Writer, String]
                                    ^

: 

In [10]:
MonadTell[Writer, String].point(3).run

cmd10.sc:1: scalaz.Writer takes two type parameters, expected: one
val res10 = MonadTell[Writer, String].point(3).run
                      ^

: 

* なんでエラーになるのかわからん

### Writer に for 構文を使う
* こうして Monad インスタンスができたので、 Writer を do 記法で自由に扱えます。

In [11]:
def logNumber(x: Int): Writer[List[String], Int] =
  x.set(List("Got number: " + x.shows))

defined function logNumber

In [12]:
def multiWithLog: Writer[List[String], Int] = for {
    a <- logNumber(3)
    b <- logNumber(5)
} yield a * b

defined function multiWithLog

In [13]:
multiWithLog.run

res12: (List[String], Int) = (List("Got number: 3", "Got number: 5"), 15)

In [14]:
def gcd(a: Int, b: Int): Writer[List[String], Int] =
  if (b == 0) for {
      _ <- List("Finished with " + a.shows).tell
    } yield a
  else
    List(a.shows + " mod" + b.shows + " = " + (a % b).shows).tell >>= { _ =>
      gcd(b, a % b)
    }

defined function gcd

In [15]:
gcd(8, 3).run

res14: (List[String], Int) = (List("8 mod3 = 2", "3 mod2 = 1", "2 mod1 = 0", "Finished with 1"), 1)

### 非効率な List の構築
* Writer モナドを使うときは、使うモナドに気をつけてください。リストを使うととても遅くなる場合があるからです。リストは mappend に ++ を使っていますが、 ++ を使ってリストの最後にものを追加する操作は、そのリストがとても長いと遅くなってしまいます。

In [16]:
Monoid[Vector[String]]

res15: Monoid[Vector[String]] = scalaz.std.VectorInstances$$anon$4@728a15c8

In [17]:
def gcd(a: Int, b: Int): Writer[Vector[String], Int] =
  if (b == 0) for {
      _ <- Vector("Finished with " + a.shows).tell
  } yield a
  else for {
      result <- gcd(b, a % b)
      _ <- Vector(a.shows + " mod" + b.shows + " = " + (a % b).shows).tell
  } yield result

defined function gcd

In [18]:
gcd(8, 3).run

res17: (Vector[String], Int) = (Vector("Finished with 1", "2 mod1 = 0", "3 mod2 = 1", "8 mod3 = 2"), 1)

### 性能の比較
* 本のように性能を比較するマイクロベンチマークを書いてみよう

In [19]:
def vectorFinalCountDown(x: Int): Writer[Vector[String], Unit] = {
    import annotation.tailrec
    @tailrec def doFinalCountDown(x: Int, w: Writer[Vector[String], Unit]): Writer[Vector[String], Unit] = x match {
        case 0 => w >>= { _ => Vector("0").tell }
        case x => doFinalCountDown(x - 1, w >>= { _ =>
        Vector(x.shows).tell
        })
    }
    val t0 = System.currentTimeMillis
    val r = doFinalCountDown(x, Vector[String]().tell)
    val t1 = System.currentTimeMillis
    r >>= { _ => Vector((t1 - t0).shows + " msec").tell }
}

def listFinalCountDown(x: Int): Writer[List[String], Unit] = {
    import annotation.tailrec
    @tailrec def doFinalCountDown(x: Int, w: Writer[List[String], Unit]): Writer[List[String], Unit] = x match {
        case 0 => w >>= { _ => List("0").tell }
        case x => doFinalCountDown(x - 1, w >>= { _ =>
        List(x.shows).tell
        })
    }
    val t0 = System.currentTimeMillis
    val r = doFinalCountDown(x, List[String]().tell)
    val t1 = System.currentTimeMillis
    r >>= { _ => List((t1 - t0).shows + " msec").tell }
}

defined function vectorFinalCountDown
defined function listFinalCountDown

In [20]:
vectorFinalCountDown(10000).run

res19: (Vector[String], Unit) = (
  Vector(
    "10000",
    "9999",
    "9998",
    "9997",
    "9996",
    "9995",
    "9994",
    "9993",
    "9992",
    "9991",
...

In [21]:
res19._1.last

res20: String = "33 msec"

In [22]:
listFinalCountDown(10000).run

res21: (List[String], Unit) = (
  List(
    "10000",
    "9999",
    "9998",
    "9997",
    "9996",
    "9995",
    "9994",
    "9993",
    "9992",
    "9991",
...

In [23]:
res21._1.last

res22: String = "961 msec"

### Reader
* 関数を作る型、 (->) r も、 Functor のインスタンスであることを見ました

In [24]:
val f = (_: Int) * 5

f: Int => Int = <function1>

In [25]:
val g = (_: Int) + 3

g: Int => Int = <function1>

In [26]:
(g map f)(8)

res25: Int = 55

* それから、関数はアプリカティブファンクターであることも見ました。これにより、関数が将来返すであろう値を、既に持っているかのように演算できるようになりました

In [27]:
val f = ({(_: Int) * 2} |@| {(_: Int) + 10}) {_ + _}

f: Int => Int = <function1>

In [28]:
f(3)

res27: Int = 19

* 関数の型 (->) r はファンクターであり、アプリカティブファンクターであるばかりでなく、モナドでもあります。これまでに登場したモナド値と同様、関数もまた文脈を持った値だとみなすことができるのです。関数にとっての文脈とは、値がまだ手元になく、値が欲しければその関数を別の何かに適用しないといけない、というものです

In [29]:
val addStuff: Int => Int = for {
    a <- (_: Int) * 2
    b <- (_: Int) + 10
} yield a + b

addStuff: Int => Int = <function1>

In [30]:
addStuff(3)

res29: Int = 19

* (*2) to (+10) はどちらも 3 に適用されます。実は、 return (a+b) も同じく 3 に適用されるんですが、引数を無視して常に a+b を返しています。そういうわけで、関数モナドは Reader モナドとも呼ばれたりします。全ての関数が共通の情報を「読む」からです。

Reader モナドは値が既にあるかのようなフリをさせてくれる。